Unzipping Dataset

In [2]:
import pickle
import numpy as np
import pandas as pd
import os

directory = '../../ppg_quality_aware_hr/data/WESAD.zip'
output_directory = '../../ppg_quality_aware_hr/data/'

if not os.path.isdir(output_directory):
    os.makedirs(output_directory)

print(os.listdir(output_directory))

import zipfile
with zipfile.ZipFile(directory, 'r') as zip_ref:
    zip_ref.extractall(output_directory)

['WESAD.zip']


Saving Individual Data

In [4]:
import pickle
import os
import numpy as np
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from ecgdetectors import Detectors
from scipy import signal
from scipy.stats import skew,kurtosis,iqr
import pickle
from peak_valley import compute_peak_valley
from respiration_feature import rip_cycle_feature_computation
filelists = ['../../ppg_quality_aware_hr/data/WESAD/'+a+'/'+a+'.pkl' for a in os.listdir('../../ppg_quality_aware_hr/data/WESAD/') if a[-1] not in ['s','f']]

In [64]:
%matplotlib notebook
def get_ecg_rr(ecg_data):
    detectors = Detectors(700)
    rpeaks = detectors.hamilton_detector(ecg_data[:,1])
    ecg_r_ts = ecg_data[np.array(rpeaks),0]
    ecg_rr_ts = ecg_r_ts[1:]
    ecg_rr_sample = np.diff(ecg_r_ts)
    ecg_rr = pd.DataFrame(np.vstack([ecg_rr_ts,ecg_rr_sample]).T,columns=['time','rr'])
    ecg_rr['timestamp'] = ecg_rr['time'].apply(lambda a:datetime.utcfromtimestamp(a))
    return ecg_rr

def bandpass_filter_ppg(data,Fs=64,fil_type='ppg'):
    X0 = data[:,1]
    X1 = signal.detrend(X0,axis=0,type='constant')
    X2 = np.zeros((np.shape(X1)[0],data.shape[1]))
    nyq = Fs/2
    b = signal.firls(219,np.array([0,0.3,0.5,3,3.5,nyq]),
                              np.array([0,0,1,1,0,0]),np.array([10,1,1]),fs=nyq*2)
    a = [1]
    X2[:,0] = data[:,0]
    X2[:,1] = signal.filtfilt(b, a, X1)
    return X2

def bandpass_filter_respiration(data,Fs=700,fil_type='ppg'):
    X0 = data[:,1]
    X1 = signal.detrend(X0,axis=0,type='constant')
    X2 = np.zeros((np.shape(X1)[0],data.shape[1]))
    nyq = Fs/2
    b = signal.firls(219,np.array([0,0.02,0.05,1,1.5,nyq]),
                              np.array([0,0,1,1,0,0]),np.array([10,1,1]),fs=nyq*2)
    a = [1]
    X2[:,0] = data[:,0]
    X2[:,1] = signal.filtfilt(b, a, X1)
    return X2

def get_quality_features(ppg_data,ppg_fs=64,window_size=2.5):
    ppg_data_final = []
    n = int(ppg_fs*window_size/2)
    for i in range(n,ppg_data.shape[0]-n,1):
        tmp = []
        tmp.append(ppg_data[i,0])
        tmp.append(ppg_data[i,1])
        sample = ppg_data[(i-n):(i+n),1]
        tmp.append(skew(sample))
        tmp.append(kurtosis(sample))
        tmp.append(iqr(sample))
        f,pxx = signal.welch(sample,fs=ppg_fs,nperseg=len(sample)//2,nfft=10000,axis=0)
        tmp.append(np.trapz(pxx[np.where((f>=.8)&(f<=2.5))[0]])/np.trapz(pxx))
        ppg_data_final.append(np.array(tmp))
    return np.array(ppg_data_final)


def save_participant_data(filename,ecg_fs = 700,ppg_fs = 64,acc_fs=32,window_size=8):
    data = pickle.load(open(filename,'rb'),encoding='latin1')
    ppg_data = data['signal']['wrist']['BVP']
    acc_data = data['signal']['wrist']['ACC']/64
    ecg_data = data['signal']['chest']['ECG']
    respiration_data = data['signal']['chest']['Resp']
    total_seconds = ppg_data.shape[0]/ppg_fs
    start_ts = datetime.utcnow().timestamp()
    ecg_ts = start_ts + np.arange(0,total_seconds,1/ecg_fs)
    acc_ts = start_ts + np.arange(0,total_seconds,1/acc_fs)
    acc_data = np.concatenate([acc_ts.reshape(-1,1),acc_data],axis=1)
    respiration_ts = ecg_ts
    respiration_data = np.vstack([respiration_ts,respiration_data.reshape(-1)]).T
    ecg_data = np.vstack([ecg_ts,ecg_data.reshape(-1)]).T
    ecg_rr1 = get_ecg_rr(ecg_data)
    ecg_rr = ecg_rr1.values
    ppg_ts = start_ts + np.arange(0,total_seconds,1/ppg_fs)
    ppg_data = np.vstack([ppg_ts,ppg_data.reshape(-1)]).T
#     ppg_data = bandpass_filter_ppg(ppg_data,Fs=ppg_fs,fil_type='ppg')
#     plt.figure()
#     plt.plot(respiration_data[:,0],respiration_data[:,1])
    respiration_data = bandpass_filter_respiration(respiration_data,Fs=ecg_fs,fil_type='ppg')
#     plt.plot(respiration_data[:,0],respiration_data[:,1])
#     plt.show()
#     return 0
    respiration_data[:,0] = respiration_data[:,0]*1000
    peak_index,valley_index = compute_peak_valley(respiration_data)
    peak_data = respiration_data[peak_index]
    valley_data = respiration_data[valley_index]
    rip_feature = rip_cycle_feature_computation(peak_data,valley_data)[:,:5]
    rip_feature[:,:2] = rip_feature[:,:2]/1000
    ppg_data = get_quality_features(ppg_data)
    ppg_data = pd.DataFrame(ppg_data,columns=['time','ppg','skew','kurtosis','iqr','relative_power']).dropna().sort_values('time').reset_index(drop=True)
#     ppg_data['timestamp'] = ppg_data['time'].apply(lambda a:datetime.utcfromtimestamp(a))
    respiration_data[:,0] = respiration_data[:,0]/1000
    all_data = []
    for i in range(0,ppg_data.shape[0]-window_size*ppg_fs,window_size*ppg_fs//4):
        a = ppg_data.loc[i:i+window_size*ppg_fs-1]
        b = respiration_data[np.where((respiration_data[:,0]>=a['time'].min())&(respiration_data[:,0]<a['time'].max()))[0],1].reshape(-1,1)
        all_data.append([a['time'].min(),a['time'].max(),
                         a[['time','ppg','skew','kurtosis','iqr','relative_power']].sort_values('time').reset_index(drop=True),b])
    
    ppg_windows = pd.DataFrame(all_data,columns=['start_time','end_time','data','respiration'])
    ppg_windows['ecg_rr'] = ppg_windows.apply(lambda a:np.mean(ecg_rr[np.where((ecg_rr[:,0]>=a['start_time'])&(ecg_rr[:,0]<a['end_time']))[0],1]),axis=1)
    ppg_windows['inspiration_duration'] = ppg_windows.apply(lambda a:np.mean(rip_feature[np.where((rip_feature[:,1]>=a['start_time'])&(rip_feature[:,0]<a['end_time']))[0],2]),axis=1)
    ppg_windows['expiration_duration'] = ppg_windows.apply(lambda a:np.mean(rip_feature[np.where((rip_feature[:,1]>=a['start_time'])&(rip_feature[:,0]<a['end_time']))[0],3]),axis=1)
    ppg_windows['respiration_duration'] = ppg_windows.apply(lambda a:np.mean(rip_feature[np.where((rip_feature[:,1]>=a['start_time'])&(rip_feature[:,0]<a['end_time']))[0],4]),axis=1)
    ppg_windows['acc_window'] = ppg_windows.apply(lambda a: acc_data[np.where((acc_data[:,0]>=a['start_time'])&(acc_data[:,0]<a['end_time']))[0],:],axis=1)
    print(ppg_windows.shape,ppg_windows.dropna().shape)
    print(ppg_windows.shape)
    if not os.path.isdir(output_directory+str(window_size)):
        os.makedirs(output_directory+str(window_size))
    final_path = output_directory+str(window_size)+'/'
    participant_name = filename.split('/')[-1]
    pickle.dump(ppg_windows,open(final_path+participant_name,'wb'))
    return ppg_windows

from joblib import Parallel,delayed
output_directory = '../../ppg_quality_aware_hr/data/'
final = Parallel(n_jobs=10,verbose=2)(delayed(save_participant_data)(f,window_size=8) for f in filelists)

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   4 out of  15 | elapsed:  7.3min remaining: 20.0min
[Parallel(n_jobs=10)]: Done  12 out of  15 | elapsed: 14.6min remaining:  3.7min
[Parallel(n_jobs=10)]: Done  15 out of  15 | elapsed: 16.3min finished


Sanity Check

In [65]:
filelists = ['../../ppg_quality_aware_hr/data/8/'+a for a in os.listdir('../../ppg_quality_aware_hr/data/8/') if a[-1] not in ['s','f']]

In [67]:
for i in range(len(filelists)):
    data = pickle.load(open(filelists[i],'rb'))
#     data['acc_window'] = data['acc_window'].apply(lambda a:a[:255])
    data['shape'] = data['data'].apply(lambda a:a.shape[0])
    print(data['shape'].unique())

#     plt.hist(data['data'].loc[0]['relative_power'])
#     plt.show()

[512]
[512]
[512]
[512]
[512]
[512]
[512]
[512]
[512]
[512]
[512]
[512]
[512]
[512]
[512]
